# Comparison of Similarity Metrics

This appendix compares the effect of all similarity metric implementations of the textdistance library [[TeDi](./A_References.ipynb#tedi)] for some example strings pair combinations for each feature to be calculated. The comparison is based on the goldstandard data and is the basis for deciding the similarity metrics for each feature to be used.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Functions for Similarity Metrics Analysis](#Functions-for-Similarity-Metrics-Analysis)
- [Similarity Metric Assessments](#Similarity-Metric-Assessments)
    - [coordinate](#coordinate)
    - [corporate](#corporate)
    - [doi](#doi)
    - [edition](#edition)
    - [exactDate](#exactDate)
    - [format](#format)
    - [isbn](#isbn)
    - [ismn](#ismn)
    - [musicid](#musicid)
    - [part](#part)
    - [person](#person)
    - [pubinit](#pubinit)
    - [scale](#scale)
    - [ttlfull](#ttlfull)
    - [volumes](#volumes)

## Data Takeover

As a first step, the training data set as a result of chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb) is read. Some sample strings of this data set will be used for the comparison and assessment of the different metrics implementations.

In [1]:
import os
import pandas as pd
import pickle as pk
import bz2
import _pickle as cPickle

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore DataFrame with features from compressed pickle file
with bz2.BZ2File((os.path.join(
    path_goldstandard, 'feature_base_df.pkl')), 'rb') as file:
    df_feature_base = cPickle.load(file)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,035liste_x,035liste_y,century_x,century_y,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,corporate_full_x,corporate_full_y,decade_x,decade_y,docid_x,docid_y,doi_x,doi_y,duplicates,edition_x,edition_y,exactDate_x,exactDate_y,format_postfix_x,format_postfix_y,format_prefix_x,format_prefix_y,isbn_x,isbn_y,ismn_x,ismn_y,masters_docid,musicid_x,musicid_y,pages_x,pages_y,part_x,part_y,person_100_x,person_100_y,person_245c_x,person_245c_y,person_700_x,person_700_y,pubinit_x,pubinit_y,pubword_x,pubword_y,pubyear_x,pubyear_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,ttlpart_x,ttlpart_y,volumes_x,volumes_y
0,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,"grawechristian, graweursula","grawechristian, graweursula",reclam jun.,reclam jun.,[Reclam jun.],[Reclam jun.],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
1,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman",emma,,,"{'245': ['Emma', 'Roman']}",{'245': ['Emma']},600,600
2,"[(OCoLC)731635279, (ABN)000539983]","[(OCoLC)731635279, (LIBIB)000315536]",2009,2009,,,,,[],[],,,,,,,2009,2009,000311049,323173349,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem englischen übersetzt von...,jane austen,"grawechristian, graweursula",,reclam jun.,reclam,[Reclam jun.],[Reclam],2009,2009,,,"emma, roman","emma, roman",,,"{'245': ['Emma', 'Roman']}","{'245': ['Emma', 'Roman']}",600,600
3,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (ABN)000539983]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,000311049,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,,"grawechristian, graweursula",reclam,reclam jun.,[Reclam],[Reclam jun.],2009,2009,,,emma,"emma, roman",,,{'245': ['Emma']},"{'245': ['Emma', 'Roman']}",600,600
4,"[(OCoLC)731635279, (NEBIS)009587153]","[(OCoLC)731635279, (NEBIS)009587153]",2009,2009,,,,,[],[],,,,,,,2009,2009,196506476,196506476,,,1,,,2009uuuu,2009uuuu,020000,020000,bk,bk,[978-3-15-020008-7],[978-3-15-020008-7],,,504389793,,,[600 S.],[600 S.],20008,20008,austenjane,austenjane,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,,,reclam,reclam,[Reclam],[Reclam],2009,2009,,,emma,emma,,,{'245': ['Emma']},{'245': ['Emma']},600,600


## Functions for Similarity Metrics Analysis

All available metrics algorithms of library textdistance [[TeDi](./A_References.ipynb#tedi)] are listed in the dictionary below. The dictionary will help calculating a similarity value for each available algorithm of the library.

In [2]:
import textdistance as tedi

tedi_algorithms = {
    # Edit based
    'Hamming' : tedi.Hamming(), 'MLIPNS' : tedi.MLIPNS(), 'Levenshtein' : tedi.Levenshtein(),
    'DamerauLevenshtein' : tedi.DamerauLevenshtein(), 'Jaro' : tedi.Jaro(), 'JaroWinkler' : tedi.JaroWinkler(),
    'StrCmp95' : tedi.StrCmp95(), 'NeedlemanWunsch' : tedi.NeedlemanWunsch(), 'Gotoh' : tedi.Gotoh(),
    'SmithWaterman' : tedi.SmithWaterman(),
    # Token based
    'Jaccard' : tedi.Jaccard(), 'Sorensen' : tedi.Sorensen(), 'Tversky' : tedi.Tversky(), 'Overlap' : tedi.Overlap(),
    'Tanimoto' : tedi.Tanimoto(), 'Cosine' : tedi.Cosine(), 'MongeElkan' : tedi.MongeElkan(), 'Bag' : tedi.Bag(),
    # Sequence based
    'LCSSeq' : tedi.LCSSeq(), 'LCSStr' : tedi.LCSStr(), 'RatcliffObershelp' : tedi.RatcliffObershelp(),
    # Compression based
    'ArithNCD' : tedi.ArithNCD(), 'RLENCD' : tedi.RLENCD(), 'BWTRLENCD' : tedi.BWTRLENCD(),
    'SqrtNCD' : tedi.SqrtNCD(), 'EntropyNCD' : tedi.EntropyNCD(), 'BZ2NCD' : tedi.BZ2NCD(),
    'LZMANCD' : tedi.LZMANCD(), 'ZLIBNCD' : tedi.ZLIBNCD(),
    # Phonetic
    'MRA' : tedi.MRA(), 'Editex' : tedi.Editex(),
    # Simple
    'Prefix' : tedi.Prefix(), 'Postfix' : tedi.Postfix(), 'Length' : tedi.Length(), 'Identity' : tedi.Identity(),
    'Matrix' : tedi.Matrix()
}

This appendix uses function $\texttt{.apply}\_\texttt{similarities}()$ that applies the $\texttt{.normalized}\_\texttt{similarity}()$ function of $\texttt{textdistance}$ for each algorithm object available in the library. The function is implemented in the separate code file [data_analysis_funcs.py](./data_analysis_funcs.py) 

In [3]:
import data_analysis_funcs as daf

## Similarity Metric Assessments

This section iterates through all available similarity metrics of library [[TeDi](./A_References.ipynb#tedi)] and calculates the similarity values for a pair of two sample strings of each feature of the model. The calculated similarity values will be analysed visually and an algorithm will be decided to be used in chapter [Feature Matrix Generation](./4_FeatureMatrixGeneration.ipynb). The decision will be based on a visual assessment that is validated with the literature, [[Chri2012](./A_References.ipynb#chri2012)].

In [4]:
# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(tedi_algorithms)+3

def num_of_samples (df) :
    max_number_of_num_samples = 30

    return min(len(df), max_number_of_num_samples)

### coordinate

In [5]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'coordinate_E_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
4,e0060811,e0060811,1.000,1.0,1.000,1.000,1.000000,1.000000,1.000000,1.0000,1.000,1.000000,1.000,1.000000,1.000,0.000000,1.000,1.000000,1.000,1.000,1.000,1.000,0.0,0.333333,0.585786,1.000000,0.892857,0.76,0.857143,1.000000,1.000,1.000,1.000,1.0,1.0,1.0
5,e0060811,e0074147,0.375,0.0,0.375,0.500,0.666667,0.666667,0.666667,0.6875,0.375,0.333333,0.500,0.333333,0.500,-1.584963,0.500,0.046875,0.500,0.500,0.375,0.500,0.0,0.111111,0.348914,0.862591,0.821429,0.84,0.428571,0.333333,0.375,0.375,0.000,1.0,0.0,0.0
2,e0060811,e0080855,0.625,1.0,0.625,0.625,0.683333,0.683333,0.683333,0.8125,0.625,0.454545,0.625,0.454545,0.625,-1.137504,0.625,0.039062,0.625,0.625,0.375,0.625,0.0,0.333333,0.371374,0.856505,0.892857,0.84,0.428571,0.666667,0.750,0.375,0.000,1.0,0.0,0.0
1,e0060811,e0055700,0.375,0.0,0.375,0.375,0.666667,0.666667,0.666667,0.6875,0.375,0.333333,0.500,0.333333,0.500,-1.584963,0.500,0.031250,0.500,0.500,0.375,0.500,0.0,0.111111,0.272049,0.795039,0.857143,0.84,0.428571,0.333333,0.500,0.375,0.000,1.0,0.0,0.0
3,e0060811,e0080851,0.750,1.0,0.750,0.750,0.777778,0.844444,0.800000,0.8750,0.750,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.054688,0.750,0.750,0.375,0.750,0.0,0.333333,0.482362,0.950679,0.892857,0.84,0.428571,0.500000,0.750,0.375,0.125,1.0,0.0,0.0
0,e0060811,,0.000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.000,0.000000,0.000,-inf,0.000,0.000000,0.000,0.000,0.000,0.000,0.0,0.111111,0.000000,0.316893,0.000000,0.36,0.428571,0.000000,0.000,0.000,0.000,0.0,0.0,0.0


### corporate

In [6]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'corporate_full_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
14,arts florissants,schweizbundesamt für landestopografie,0.000000,0.0,0.162162,0.162162,0.593079,0.593079,0.619937,0.297297,0.125000,0.394737,0.566038,0.394737,0.937500,-1.341037,0.616496,0.031250,0.405405,0.216216,0.054054,0.301887,0.0,0.000000,0.275398,0.804628,0.403226,0.512195,0.142857,0.000000,0.270270,0.0,0.000000,0.432432,0.0,0.0
22,arts florissants,eidgenössische landestopographie,0.062500,0.0,0.156250,0.187500,0.555871,0.555871,0.593371,0.328125,0.062500,0.371429,0.541667,0.371429,0.812500,-1.428843,0.574524,0.029297,0.406250,0.187500,0.062500,0.250000,0.0,0.000000,0.303292,0.846177,0.452830,0.538462,0.230769,0.000000,0.265625,0.0,0.000000,0.500000,0.0,0.0
25,arts florissants,oper (köln),0.062500,0.0,0.125000,0.125000,0.455682,0.455682,0.486364,0.406250,0.090909,0.227273,0.370370,0.227273,0.454545,-2.137504,0.376889,0.011719,0.312500,0.250000,0.062500,0.296296,0.0,0.000000,0.247112,0.840105,0.589744,0.724138,0.272727,0.000000,0.250000,0.0,0.000000,0.687500,0.0,0.0
7,arts florissants,schweizerische gesellschaft für bildungsforsch...,0.030000,0.0,0.120000,0.120000,0.552619,0.552619,0.559869,0.140000,0.125000,0.160000,0.275862,0.160000,1.000000,-2.643856,0.400000,0.031250,0.160000,0.120000,0.030000,0.155172,0.0,0.022472,0.200135,0.810645,0.241758,0.365079,0.170732,0.166667,0.255000,0.0,0.000000,0.160000,0.0,0.0
21,arts florissants,interkantonale lehrmittelzentrale (luzern),0.023810,0.0,0.166667,0.190476,0.498286,0.498286,0.524179,0.273810,0.062500,0.234043,0.379310,0.234043,0.687500,-2.095157,0.424334,0.021484,0.261905,0.190476,0.071429,0.137931,0.0,0.024390,0.263176,0.853271,0.433962,0.488372,0.217391,0.166667,0.250000,0.0,0.000000,0.380952,0.0,0.0
30,arts florissants,springerlink (online service),0.034483,0.0,0.172414,0.172414,0.476133,0.476133,0.505228,0.362069,0.000000,0.250000,0.400000,0.250000,0.562500,-2.000000,0.417815,0.021484,0.310345,0.206897,0.068966,0.177778,0.0,0.000000,0.267984,0.844239,0.478261,0.567568,0.181818,0.000000,0.241379,0.0,0.000000,0.551724,0.0,0.0
36,arts florissants,trägerverein 600 jahre niklaus von flüe 1417-2017,0.040816,0.0,0.122449,0.122449,0.476361,0.476361,0.492942,0.224490,0.000000,0.226415,0.369231,0.226415,0.750000,-2.142958,0.428571,0.031250,0.244898,0.122449,0.061224,0.184615,0.0,0.020000,0.215327,0.766007,0.350649,0.446809,0.122807,0.166667,0.193878,0.0,0.000000,0.326531,0.0,0.0
11,arts florissants,"staatsoper (wien)chor, wiener philharmoniker",0.000000,0.0,0.204545,0.204545,0.535354,0.535354,0.552399,0.284091,0.125000,0.276596,0.433333,0.276596,0.812500,-1.854149,0.489956,0.029297,0.295455,0.204545,0.045455,0.133333,0.0,-0.023256,0.261027,0.820436,0.406780,0.441860,0.152174,0.166667,0.261364,0.0,0.000000,0.363636,0.0,0.0
32,arts florissants,suisse,0.062500,0.0,0.250000,0.250000,0.638889,0.638889,0.684722,0.312500,0.166667,0.222222,0.363636,0.222222,0.666667,-2.169925,0.408248,0.009766,0.250000,0.250000,0.187500,0.363636,0.0,0.062500,0.136579,0.658166,0.571429,0.724138,0.363636,0.000000,0.312500,0.0,0.000000,0.375000,0.0,0.0
9,arts florissants,"bonne pioche (paris), buena vista internationa...",0.000000,0.0,0.156250,0.156250,0.506944,0.506944,0.522569,0.203125,0.187500,0.230769,0.375000,0.230769,0.937500,-2.115477,0.468750,0.031250,0.234375,0.156250,0.046875,0.225000,0.0,0.000000,0.240263,0.813623,0.356164,0.396226,0.234375,0.166667,0.218750,0.0,0.015625,0.250000,0.0,0.0


Monge-Elkan, Jaccard with q-grams, and LCSStr seem to be valid metrics for the $\texttt{corporate}$ attribute due to their algorithms [[Chri2012](./A_References.ipynb#chri2012)]. The metrics to be chosen will be analysed and justified in chapter [Feature Matrix Generation](./4_FeatureMatrixGeneration.ipynb).

### doi

For attribute $\texttt{doi}$, a preprocessing function has been implemented to extract real doi identifiers, see chapter [Data Analysis](./1_DataAnalysis.ipynb). The DataFrame $\texttt{df}\_\texttt{feature}\_\texttt{base}$ holds doi identifiers as preprocessed singular strings.

In [7]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'doi_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
4,10.5167/uzh-53042,10.1007/978-3-642-41698-9,0.320000,0.0,0.320000,0.400000,0.700850,0.790595,0.700850,0.500000,0.000000,0.400000,0.571429,0.400000,0.705882,-1.321928,0.582086,0.020761,0.480000,0.400000,0.120000,0.285714,0.0,0.115385,0.413811,0.917367,0.658537,0.657143,0.258065,0.5,0.360000,0.120000,0.0,0.680000,0.0,0.0
6,10.5167/uzh-53042,10.1093/cid/ciu795,0.222222,0.0,0.222222,0.277778,0.555011,0.555011,0.555011,0.583333,0.176471,0.400000,0.571429,0.400000,0.588235,-1.321928,0.571662,0.019031,0.555556,0.388889,0.166667,0.228571,0.0,0.052632,0.351929,0.894990,0.619048,0.741935,0.260870,0.5,0.250000,0.166667,0.0,0.944444,0.0,0.0
0,10.5167/uzh-53042,,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.055556,0.000000,0.211215,0.000000,0.290323,0.260870,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
3,10.5167/uzh-53042,10.1055/b-002-26639,0.210526,0.0,0.263158,0.315789,0.620790,0.620790,0.631935,0.578947,0.176471,0.500000,0.666667,0.500000,0.705882,-1.000000,0.667698,0.020761,0.631579,0.421053,0.157895,0.333333,0.0,0.050000,0.421543,0.908927,0.675000,0.741935,0.240000,0.5,0.368421,0.157895,0.0,0.894737,0.0,0.0
9,10.5167/uzh-53042,10.1055/b-005-143650,0.300000,0.0,0.300000,0.350000,0.611141,0.611141,0.611141,0.575000,0.235294,0.480000,0.648649,0.480000,0.705882,-1.058894,0.650791,0.020761,0.600000,0.450000,0.150000,0.432432,0.0,0.047619,0.425815,0.885333,0.650000,0.677419,0.269231,0.5,0.400000,0.150000,0.0,0.850000,0.0,0.0
2,10.5167/uzh-53042,10.1093/cid/cir669,0.222222,0.0,0.222222,0.277778,0.506536,0.506536,0.517974,0.583333,0.176471,0.296296,0.457143,0.296296,0.470588,-1.754888,0.457330,0.013841,0.444444,0.277778,0.166667,0.285714,0.0,0.105263,0.275067,0.859452,0.634146,0.741935,0.260870,0.5,0.277778,0.166667,0.0,0.944444,0.0,0.0
5,10.5167/uzh-53042,10.1093/ndt/gft319,0.222222,0.0,0.222222,0.277778,0.506536,0.506536,0.506536,0.583333,0.176471,0.250000,0.400000,0.250000,0.411765,-2.000000,0.400163,0.012111,0.388889,0.333333,0.166667,0.342857,0.0,0.052632,0.242001,0.848238,0.585366,0.741935,0.250000,0.5,0.250000,0.166667,0.0,0.944444,0.0,0.0
8,10.5167/uzh-53042,10.5451/unibas-006499413,0.250000,0.0,0.333333,0.375000,0.651961,0.651961,0.662010,0.520833,0.058824,0.464286,0.634146,0.464286,0.764706,-1.106915,0.643596,0.022491,0.541667,0.416667,0.166667,0.439024,0.0,0.040000,0.362457,0.932628,0.595745,0.636364,0.233333,0.5,0.416667,0.166667,0.0,0.708333,0.0,0.0
7,10.5167/uzh-53042,10.5451/unibas-006503313,0.250000,0.0,0.375000,0.375000,0.651961,0.651961,0.662010,0.541667,0.117647,0.464286,0.634146,0.464286,0.764706,-1.106915,0.643596,0.022491,0.541667,0.416667,0.166667,0.487805,0.0,0.040000,0.386449,0.943422,0.608696,0.636364,0.233333,0.5,0.416667,0.166667,0.0,0.708333,0.0,0.0
1,10.5167/uzh-53042,10.5167/uzh-53042,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.055556,0.585786,1.000000,0.875000,0.870968,0.869565,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0


### edition

In [8]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'edition_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
5,8,1791,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.125,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.200000,0.000000,0.231229,0.869565,0.913043,0.600000,0.0,0.00,0.0,0.0,0.25,0.0,0.0
4,8,2,0.0,1.0,0.00,0.00,0.00,0.00,0.00,0.500,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.500000,0.000000,0.000000,1.000000,1.000000,0.857143,0.0,0.00,0.0,0.0,1.00,0.0,0.0
1,8,1,0.0,1.0,0.00,0.00,0.00,0.00,0.00,0.500,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.500000,0.000000,0.000000,1.000000,1.000000,0.857143,0.0,0.00,0.0,0.0,1.00,0.0,0.0
10,8,10,0.0,1.0,0.00,0.00,0.00,0.00,0.00,0.250,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.333333,0.000000,0.207519,0.954545,1.000000,0.750000,0.0,0.00,0.0,0.0,0.50,0.0,0.0
2,8,10425,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.100,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.166667,0.000000,0.221849,0.880000,0.920000,0.545455,0.0,0.00,0.0,0.0,0.20,0.0,0.0
13,8,7,0.0,1.0,0.00,0.00,0.00,0.00,0.00,0.500,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.500000,0.000000,0.000000,1.000000,1.000000,0.857143,0.0,0.00,0.0,0.0,1.00,0.0,0.0
0,8,,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.000,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.500000,0.000000,1.000000,0.000000,0.391304,0.857143,0.0,0.00,0.0,0.0,0.00,0.0,0.0
8,8,3,0.0,1.0,0.00,0.00,0.00,0.00,0.00,0.500,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.500000,0.000000,0.000000,1.000000,1.000000,0.857143,0.0,0.00,0.0,0.0,1.00,0.0,0.0
7,8,8,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.000,1.0,1.00,1.0,1.00,1.0,0.0,1.0,1.0,1.00,1.00,1.00,1.0,0.0,0.500000,0.585786,1.000000,1.000000,1.000000,0.857143,1.0,1.00,1.0,1.0,1.00,1.0,1.0
6,8,19,0.0,1.0,0.00,0.00,0.00,0.00,0.00,0.250,0.0,0.00,0.0,0.00,0.0,-inf,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.333333,0.000000,0.207519,0.954545,1.000000,0.750000,0.0,0.00,0.0,0.0,0.50,0.0,0.0


### exactDate

In [9]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'exactDate_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
Gotoh
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,Gotoh,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
90,19949999,1862uuuu,0.125,0.0,0.125,0.125,0.416667,0.416667,0.416667,0.5625,0.5625,0.125,0.066667,0.125,0.066667,0.125,-3.906891,0.125,0.007812,0.125,0.125,0.125,0.125,0.0,0.000000,0.097631,0.551880,0.766667,0.84,0.416667,0.250000,0.500,0.125,0.000,1.0,0.0,0.0
2,19949999,1999uuuu,0.375,0.0,0.375,0.500,0.666667,0.666667,0.666667,0.6875,0.6875,0.375,0.333333,0.500,0.333333,0.500,-1.584963,0.500,0.054688,0.500,0.500,0.375,0.500,0.0,0.250000,0.373480,0.779419,0.800000,0.76,0.583333,0.500000,0.750,0.375,0.000,1.0,0.0,0.0
75,19949999,183uuuuu,0.125,0.0,0.125,0.125,0.416667,0.416667,0.416667,0.5625,0.5625,0.125,0.066667,0.125,0.066667,0.125,-3.906891,0.125,0.007812,0.125,0.125,0.125,0.125,0.0,0.000000,0.111875,0.561064,0.766667,0.84,0.416667,0.250000,0.625,0.125,0.000,1.0,0.0,0.0
77,19949999,19911794,0.500,0.0,0.500,0.500,0.666667,0.666667,0.666667,0.7500,0.7500,0.500,0.454545,0.625,0.454545,0.625,-1.137504,0.625,0.062500,0.625,0.500,0.375,0.500,0.0,-0.111111,0.457418,0.811246,0.785714,0.84,0.500000,0.333333,0.625,0.375,0.000,1.0,0.0,0.0
61,19949999,19739999,0.750,1.0,0.750,0.750,0.833333,0.866667,0.833333,0.8750,0.8750,0.750,0.600000,0.750,0.600000,0.750,-0.736966,0.750,0.054688,0.750,0.750,0.500,0.750,0.0,0.222222,0.373318,0.829189,0.821429,0.84,0.500000,0.400000,0.750,0.250,0.500,1.0,0.0,0.0
26,19949999,18uuuuuu,0.125,0.0,0.125,0.125,0.416667,0.416667,0.416667,0.5625,0.5625,0.125,0.066667,0.125,0.066667,0.125,-3.906891,0.125,0.007812,0.125,0.125,0.125,0.125,0.0,0.000000,0.131652,0.575506,0.793103,0.76,0.416667,0.250000,0.625,0.125,0.000,1.0,0.0,0.0
28,19949999,2010uuuu,0.000,0.0,0.000,0.000,0.416667,0.416667,0.416667,0.5000,0.5000,0.000,0.066667,0.125,0.066667,0.125,-3.906891,0.125,0.007812,0.125,0.125,0.125,0.125,0.0,0.250000,0.108194,0.556596,0.800000,0.84,0.416667,0.000000,0.375,0.000,0.000,1.0,0.0,0.0
80,19949999,1764uuuu,0.250,0.0,0.250,0.250,0.500000,0.500000,0.500000,0.6250,0.6250,0.250,0.142857,0.250,0.142857,0.250,-2.807355,0.250,0.015625,0.250,0.250,0.125,0.250,0.0,0.000000,0.195262,0.593546,0.800000,0.84,0.416667,0.250000,0.625,0.125,0.000,1.0,0.0,0.0
15,19949999,2005uuuu,0.000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.5000,0.5000,0.000,0.000000,0.000,0.000000,0.000,-inf,0.000,0.000000,0.000,0.000,0.000,0.000,0.0,0.000000,0.000000,0.511141,0.766667,0.84,0.416667,0.000000,0.500,0.000,0.000,1.0,0.0,0.0
74,19949999,1836uuuu,0.125,0.0,0.125,0.125,0.416667,0.416667,0.416667,0.5625,0.5625,0.125,0.066667,0.125,0.066667,0.125,-3.906891,0.125,0.007812,0.125,0.125,0.125,0.125,0.0,0.000000,0.097631,0.551880,0.766667,0.84,0.416667,0.250000,0.500,0.125,0.000,1.0,0.0,0.0


Attribute $\texttt{exactDate}$ is a string of four digits or characters. For calculating the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance), each string pair is compared character-wise. A so called edit distance between the string pair is calculated as the sum of all edit operations needed to convert the strings into each other [[Chri2012](./A_References.ipynb#chri2012)]. The resulting Hamming similarity can be deduced from the edit distance and the length of one string. This can be easily done in the examples of the DataFrame above. The Hamming similarity shall be used for attribute $\texttt{exactDate}$.

The Hamming similarity has one drawbak, though, looking at Swissbib's data. The attribute may be filled with letter 'u' for 'unknown' digits instead of a number. A letter 'u' will result in an edit distance of 1. This is a statement which need not be true for the bibliographic units that the records describe. On the other hand, a pair of strings with a letter 'u' at the same digit, need not have a distance of 0 for the two bibliographical units compared. For this reason, the Hamming similarity will be adapted for the case of existence of letter 'u' in one of the strings of the pair. The Hamming similarity will be increased by a small value for each unknown digit in a string. This algorithm based on the Hamming similarity will be explicitly implemented in chapter [Feature Matrix Generation](./4_FeatureMatrixGeneration.ipynb).

### format

In [10]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'format_prefix_x')

for algorithm in tedi_algorithms :
    if algorithm not in [] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
Gotoh
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
ArithNCD
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,Gotoh,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,ArithNCD,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
3,vm,cr,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.000000,0.0,0.000000,0.0,-inf,0.0,0.000,0.0,0.0,0.0,0.0,-0.666667,0.0,0.333333,0.000000,0.50,0.875000,1.0,0.75,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0,vm,bk,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.000000,0.0,0.000000,0.0,-inf,0.0,0.000,0.0,0.0,0.0,0.0,-0.333333,0.0,0.333333,0.000000,0.50,0.791667,1.0,0.75,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,vm,vm,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.000000,1.0,0.000000,1.0,1.000,1.0,1.0,1.0,1.0,0.000000,0.0,0.333333,0.585786,1.00,1.000000,1.0,0.75,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,vm,mp,0.0,1.0,0.0,0.5,0.0,0.0,0.0,0.5,0.5,0.0,0.333333,0.5,0.333333,0.5,-1.584963,0.5,0.125,0.5,0.5,0.5,0.5,-0.333333,0.0,0.333333,0.292893,0.75,0.958333,1.0,0.75,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,vm,cf,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.000000,0.0,0.000000,0.0,-inf,0.0,0.000,0.0,0.0,0.0,0.0,-0.333333,0.0,0.333333,0.000000,0.50,0.791667,1.0,0.75,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,vm,mu,0.0,1.0,0.0,0.5,0.0,0.0,0.0,0.5,0.5,0.0,0.333333,0.5,0.333333,0.5,-1.584963,0.5,0.125,0.5,0.5,0.5,0.5,-0.333333,0.0,0.333333,0.292893,0.75,0.958333,1.0,0.75,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'format_postfix_x')

for algorithm in tedi_algorithms :
    if algorithm not in [] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
Gotoh
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
ArithNCD
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,Gotoh,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,ArithNCD,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
2,030600,040100,0.666667,1.0,0.666667,0.666667,0.777778,0.800000,0.777778,0.833333,0.833333,0.666667,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.055556,0.666667,0.666667,0.333333,0.666667,-0.714286,0.166667,0.428571,0.292893,0.851959,0.791667,0.92,0.500000,0.600000,0.666667,0.166667,0.333333,1.0,0.0,0.0
12,030600,010000,0.666667,1.0,0.666667,0.833333,0.777778,0.800000,0.777778,0.833333,0.833333,0.666667,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.055556,0.666667,0.666667,0.333333,0.500000,-0.833333,0.166667,0.285714,0.309017,0.752403,0.769231,0.92,0.583333,0.400000,0.666667,0.166667,0.333333,1.0,0.0,0.0
20,030600,030653,0.666667,1.0,0.666667,0.666667,0.777778,0.866667,0.777778,0.833333,0.833333,0.666667,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.083333,0.666667,0.666667,0.666667,0.666667,-0.500000,0.166667,0.142857,0.462402,0.836225,0.800000,0.92,0.666667,0.666667,0.666667,0.666667,0.000000,1.0,0.0,0.0
17,030600,040000,0.666667,1.0,0.666667,0.833333,0.777778,0.800000,0.777778,0.833333,0.833333,0.666667,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.055556,0.666667,0.666667,0.333333,0.500000,-1.000000,0.166667,0.285714,0.309017,0.752403,0.807692,0.92,0.583333,0.400000,0.666667,0.166667,0.333333,1.0,0.0,0.0
8,030600,020047,0.333333,0.0,0.333333,0.500000,0.666667,0.666667,0.666667,0.666667,0.666667,0.333333,0.333333,0.500000,0.333333,0.500000,-1.584963,0.500000,0.055556,0.500000,0.500000,0.333333,0.500000,-0.666667,0.166667,0.285714,0.229562,0.750863,0.846154,0.92,0.500000,0.400000,0.500000,0.166667,0.000000,1.0,0.0,0.0
1,030600,010300,0.666667,1.0,0.666667,0.666667,0.822222,0.840000,0.822222,0.833333,0.833333,0.666667,0.714286,0.833333,0.714286,0.833333,-0.485427,0.833333,0.069444,0.833333,0.666667,0.500000,0.666667,-0.666667,0.166667,0.428571,0.439340,0.925980,0.875000,0.92,0.666667,0.600000,0.666667,0.166667,0.333333,1.0,0.0,0.0
14,030600,020800,0.666667,1.0,0.666667,0.666667,0.777778,0.800000,0.777778,0.833333,0.833333,0.666667,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.055556,0.666667,0.666667,0.333333,0.666667,-0.833333,0.166667,0.428571,0.292893,0.851959,0.875000,0.92,0.500000,0.600000,0.666667,0.166667,0.333333,1.0,0.0,0.0
3,030600,030600,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.166667,0.166667,0.428571,0.585786,1.000000,0.958333,0.92,0.833333,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0
19,030600,030053,0.500000,1.0,0.500000,0.666667,0.777778,0.844444,0.777778,0.750000,0.750000,0.500000,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.069444,0.666667,0.666667,0.500000,0.666667,-0.833333,0.166667,0.428571,0.426519,0.878222,0.840000,0.92,0.500000,0.600000,0.666667,0.500000,0.000000,1.0,0.0,0.0
9,030600,020053,0.333333,0.0,0.333333,0.500000,0.666667,0.666667,0.666667,0.666667,0.666667,0.333333,0.500000,0.666667,0.500000,0.666667,-1.000000,0.666667,0.069444,0.666667,0.500000,0.333333,0.500000,-1.333333,0.166667,0.285714,0.353353,0.810547,0.840000,0.92,0.500000,0.400000,0.500000,0.166667,0.000000,1.0,0.0,0.0


### isbn

Attribute $\texttt{isbn}$ is treated as a list of string elements, see chapter [Feature Matrix Generation](./4_FeatureMatrixGeneration.ipynb). The metrics comparison will be omitted, here.

### ismn

In [12]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'ismn_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
7,m006546749,m006204687,0.400000,0.0,0.500000,0.500000,0.800000,0.880000,0.820000,0.700000,0.4,0.538462,0.700000,0.538462,0.7,-0.893085,0.700000,0.040,0.700000,0.700000,0.400000,0.700000,0.0,0.090909,0.435826,0.930879,0.800000,0.851852,0.500000,0.500000,0.500000,0.400000,0.0,1.000000,0.0,0.0
3,m006546749,m006546756,0.800000,1.0,0.800000,0.800000,0.866667,0.920000,0.880000,0.900000,0.8,0.666667,0.800000,0.666667,0.8,-0.584963,0.800000,0.045,0.800000,0.800000,0.800000,0.800000,0.0,0.090909,0.518586,0.941020,0.833333,0.777778,0.750000,0.666667,0.800000,0.800000,0.0,1.000000,0.0,0.0
6,m006546749,"m006546756 (kritischer bericht, leinen)",0.205128,0.0,0.205128,0.205128,0.668376,0.668376,0.668376,0.230769,0.0,0.195122,0.326531,0.195122,0.8,-2.357552,0.405096,0.045,0.205128,0.205128,0.205128,0.326531,0.0,0.025000,0.154418,0.705288,0.379310,0.512195,0.266667,0.500000,0.256410,0.205128,0.0,0.256410,0.0,0.0
0,m006546749,,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,-inf,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.090909,0.000000,0.268678,0.000000,0.333333,0.375000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
5,m006546749,m200205343,0.300000,0.0,0.300000,0.300000,0.666667,0.666667,0.666667,0.650000,0.3,0.333333,0.500000,0.333333,0.5,-1.584963,0.500000,0.030,0.500000,0.500000,0.200000,0.500000,0.0,0.090909,0.335321,0.850276,0.766667,0.851852,0.375000,0.333333,0.300000,0.100000,0.0,1.000000,0.0,0.0
2,m006546749,9790006450510,0.076923,0.0,0.230769,0.384615,0.561538,0.561538,0.561538,0.500000,0.3,0.437500,0.608696,0.437500,0.7,-1.192645,0.613941,0.045,0.538462,0.307692,0.230769,0.347826,0.0,0.071429,0.476471,0.931337,0.766667,0.793103,0.421053,0.000000,0.307692,0.000000,0.0,0.769231,0.0,0.0
1,m006546749,m006450510,0.400000,0.0,0.400000,0.500000,0.677778,0.677778,0.677778,0.700000,0.4,0.428571,0.600000,0.428571,0.6,-1.222392,0.600000,0.040,0.600000,0.500000,0.400000,0.500000,0.0,0.090909,0.436389,0.889499,0.833333,0.851852,0.500000,0.500000,0.400000,0.400000,0.0,1.000000,0.0,0.0
4,m006546749,m006546749,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.000000,1.000000,1.000,1.000000,1.000000,1.000000,1.000000,0.0,0.090909,0.585786,1.000000,0.900000,0.851852,0.875000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.0


### musicid

In [13]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'musicid_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
24,99064,134,0.000000,0.0,0.200000,0.200000,0.000000,0.000000,0.000000,0.400000,0.333333,0.142857,0.250000,0.142857,0.333333,-2.807355,0.258199,0.02,0.200000,0.200000,0.200000,0.250000,0.0,0.166667,0.132705,0.686838,0.840000,0.92,0.545455,0.000000,0.400000,0.0,0.2,0.600000,0.0,0.0
29,99064,50999,0.000000,0.0,0.000000,0.200000,0.433333,0.433333,0.433333,0.500000,0.000000,0.428571,0.600000,0.428571,0.600000,-1.222392,0.600000,0.06,0.600000,0.400000,0.400000,0.400000,0.4,0.500000,0.338901,0.798074,0.800000,0.92,0.818182,0.250000,0.400000,0.0,0.0,1.000000,0.0,0.0
19,99064,502430,0.000000,0.0,0.166667,0.333333,0.577778,0.577778,0.577778,0.416667,0.000000,0.222222,0.363636,0.222222,0.400000,-2.169925,0.365148,0.04,0.333333,0.333333,0.166667,0.363636,0.0,0.142857,0.234189,0.771928,0.840000,0.92,0.500000,0.333333,0.333333,0.0,0.0,0.833333,0.0,0.0
15,99064,3714,0.000000,0.0,0.200000,0.200000,0.483333,0.483333,0.483333,0.500000,0.250000,0.125000,0.222222,0.125000,0.250000,-3.000000,0.223607,0.02,0.200000,0.200000,0.200000,0.222222,0.0,0.166667,0.132705,0.732149,0.920000,0.84,0.545455,0.250000,0.400000,0.0,0.2,0.800000,0.0,0.0
22,99064,5944,0.200000,0.0,0.400000,0.600000,0.633333,0.633333,0.633333,0.600000,0.250000,0.285714,0.444444,0.285714,0.500000,-1.807355,0.447214,0.06,0.400000,0.400000,0.200000,0.444444,0.0,0.166667,0.309076,0.790110,0.840000,0.84,0.545455,0.000000,0.400000,0.0,0.2,0.800000,0.0,0.0
11,99064,245,0.000000,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.300000,0.000000,0.142857,0.250000,0.142857,0.333333,-2.807355,0.258199,0.02,0.200000,0.200000,0.200000,0.250000,0.0,0.166667,0.132705,0.686838,0.880000,0.92,0.545455,0.000000,0.200000,0.0,0.0,0.600000,0.0,0.0
3,99064,501326,0.000000,0.0,0.000000,0.166667,0.577778,0.577778,0.577778,0.416667,0.000000,0.222222,0.363636,0.222222,0.400000,-2.169925,0.365148,0.04,0.333333,0.333333,0.166667,0.363636,0.0,0.142857,0.195262,0.723275,0.800000,0.92,0.500000,0.166667,0.166667,0.0,0.0,0.833333,0.0,0.0
4,99064,502023,0.000000,0.0,0.166667,0.166667,0.455556,0.455556,0.455556,0.500000,0.200000,0.100000,0.181818,0.100000,0.200000,-3.321928,0.182574,0.02,0.166667,0.166667,0.166667,0.181818,0.0,0.142857,0.141281,0.744950,0.880000,0.92,0.500000,0.166667,0.166667,0.0,0.0,0.833333,0.0,0.0
7,99064,0171433,0.142857,0.0,0.142857,0.142857,0.561905,0.561905,0.561905,0.428571,0.000000,0.200000,0.333333,0.200000,0.400000,-2.321928,0.338062,0.04,0.285714,0.285714,0.142857,0.333333,0.0,0.125000,0.201010,0.743597,0.846154,0.92,0.461538,0.000000,0.285714,0.0,0.0,0.714286,0.0,0.0
10,99064,422,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.142857,0.250000,0.142857,0.333333,-2.807355,0.258199,0.02,0.200000,0.200000,0.200000,0.250000,0.0,0.166667,0.132705,0.544238,0.800000,0.92,0.545455,0.000000,0.200000,0.0,0.0,0.600000,0.0,0.0


### part

In [14]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'part_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
50,33001,28 10 2013 10 2421 2431,0.000000,0.0,0.130435,0.130435,0.574638,0.574638,0.574638,0.173913,0.200000,0.217391,0.357143,0.217391,1.000000,-2.201634,0.466252,0.10,0.217391,0.130435,0.086957,0.214286,0.000000,0.045455,2.058250e-01,0.688456,0.617647,0.636364,0.250000,0.000000,0.130435,0.000000,0.043478,0.217391,0.0,0.0
19,33001,27 597 27 597,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.192308,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,-2.220446e-16,0.575218,0.655172,0.793103,0.466667,0.000000,0.000000,0.000000,0.000000,0.384615,0.0,0.0
48,33001,5 5,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,1.110223e-16,0.471950,0.833333,0.920000,0.545455,0.000000,0.400000,0.000000,0.000000,0.600000,0.0,0.0
31,33001,4 4,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,1.110223e-16,0.471950,0.833333,0.920000,0.545455,0.000000,0.400000,0.000000,0.000000,0.600000,0.0,0.0
2,33001,57,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,1.110223e-16,0.509928,0.833333,0.920000,0.545455,0.000000,0.400000,0.000000,0.000000,0.400000,0.0,0.0
45,33001,3,0.200000,0.0,0.200000,0.200000,0.733333,0.733333,0.733333,0.200000,0.000000,0.200000,0.333333,0.200000,1.000000,-2.321928,0.447214,0.04,0.200000,0.200000,0.200000,0.333333,0.200000,0.166667,1.781836e-01,0.421416,0.916667,0.920000,0.545455,0.333333,0.600000,0.200000,0.000000,0.200000,0.0,0.0
52,33001,60 3 2015 02 01 432 437,0.000000,0.0,0.173913,0.173913,0.574638,0.574638,0.574638,0.195652,0.000000,0.217391,0.357143,0.217391,1.000000,-2.201634,0.466252,0.10,0.217391,0.173913,0.086957,0.214286,0.000000,0.086957,1.860777e-01,0.654602,0.567568,0.696970,0.222222,0.000000,0.173913,0.000000,0.000000,0.217391,0.0,0.0
54,33001,3870,0.400000,0.0,0.400000,0.400000,0.633333,0.633333,0.633333,0.600000,0.250000,0.285714,0.444444,0.285714,0.500000,-1.807355,0.447214,0.08,0.400000,0.400000,0.200000,0.444444,0.000000,0.166667,3.410814e-01,0.802882,0.880000,0.840000,0.545455,0.250000,0.400000,0.200000,0.000000,0.800000,0.0,0.0
39,33001,2620 5,0.166667,0.0,0.166667,0.166667,0.455556,0.455556,0.455556,0.500000,0.200000,0.100000,0.181818,0.100000,0.200000,-3.321928,0.182574,0.04,0.166667,0.166667,0.166667,0.181818,0.000000,0.142857,1.259948e-01,0.648913,0.777778,0.920000,0.500000,0.000000,0.166667,0.000000,0.000000,0.833333,0.0,0.0
44,33001,33001,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,0.000000,0.500000,5.857864e-01,1.000000,0.958333,0.920000,0.818182,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


### person

Attribute $\texttt{person}$ consists of three different representations of data, see chapter [Data Analysis](./1_DataAnalysis.ipynb). The similarity metric may be sensitive, depending on the kind of representation. All three representations will be investigated below.

In [15]:
person_representations = ['100', '245c', '700']

for pr in person_representations :
    df_string_pairs = daf.string_pair_list(df_feature_base, 'person_'+pr+'_x')

    print('\nperson_'+pr+'\n**********')
    for algorithm in tedi_algorithms :
        if algorithm not in ['Gotoh', 'ArithNCD'] :
            daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

    display(df_string_pairs.sample(n=num_of_samples(df_string_pairs)))


person_100
**********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
24,eigenmanndaniela,eigenmanndaniela,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.294118,0.585786,1.000000,0.875000,0.793103,0.863636,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0
8,eigenmanndaniela,frischmax,0.000000,0.0,0.125000,0.125000,0.506944,0.506944,0.541667,0.312500,0.000000,0.136364,0.240000,0.136364,0.333333,-2.874469,0.250000,0.011719,0.187500,0.187500,0.125000,0.240000,0.0,0.058824,0.183852,0.817065,0.687500,0.724138,0.272727,0.000000,0.250000,0.0,0.000000,0.562500,0.0,0.0
5,eigenmanndaniela,gläser-zikudamichaela,0.000000,0.0,0.333333,0.333333,0.646194,0.646194,0.657206,0.547619,0.187500,0.423077,0.594595,0.423077,0.687500,-1.241008,0.600099,0.023438,0.523810,0.380952,0.142857,0.432432,0.0,0.045455,0.288893,0.800673,0.416667,0.636364,0.250000,0.166667,0.404762,0.0,0.142857,0.761905,0.0,0.0
26,eigenmanndaniela,schneiderreto u.,0.000000,0.0,0.000000,0.062500,0.472222,0.472222,0.509722,0.500000,0.000000,0.230769,0.375000,0.230769,0.375000,-2.115477,0.375000,0.019531,0.375000,0.250000,0.125000,0.250000,0.0,0.058824,0.215357,0.792543,0.540541,0.724138,0.272727,0.000000,0.281250,0.0,0.000000,1.000000,0.0,0.0
21,eigenmanndaniela,dufourguillaume henri,0.000000,0.0,0.095238,0.142857,0.515542,0.515542,0.559590,0.428571,0.062500,0.370370,0.540541,0.370370,0.625000,-1.432959,0.545545,0.031250,0.476190,0.190476,0.095238,0.216216,0.0,0.045455,0.340648,0.825525,0.585366,0.636364,0.259259,0.166667,0.357143,0.0,0.000000,0.761905,0.0,0.0
1,eigenmanndaniela,levinejamesdir.,0.125000,0.0,0.187500,0.250000,0.609722,0.609722,0.622639,0.531250,0.066667,0.476190,0.645161,0.476190,0.666667,-1.070389,0.645497,0.029297,0.625000,0.375000,0.062500,0.387097,0.0,0.058824,0.387668,0.867822,0.631579,0.724138,0.272727,0.333333,0.375000,0.0,0.000000,0.937500,0.0,0.0
19,eigenmanndaniela,bruchjulia,0.000000,0.0,0.125000,0.125000,0.384722,0.384722,0.449722,0.375000,0.100000,0.130435,0.230769,0.130435,0.300000,-2.938599,0.237171,0.011719,0.187500,0.125000,0.062500,0.153846,0.0,0.058824,0.183852,0.809601,0.656250,0.724138,0.272727,0.166667,0.343750,0.0,0.062500,0.625000,0.0,0.0
0,eigenmanndaniela,austenjane,0.062500,0.0,0.250000,0.250000,0.602778,0.602778,0.635278,0.437500,0.200000,0.300000,0.461538,0.300000,0.600000,-1.736966,0.474342,0.019531,0.375000,0.312500,0.125000,0.384615,0.0,0.058824,0.256025,0.891021,0.625000,0.724138,0.272727,0.000000,0.437500,0.0,0.000000,0.625000,0.0,0.0
12,eigenmanndaniela,binderheidy,0.125000,0.0,0.187500,0.250000,0.529040,0.529040,0.544381,0.437500,0.181818,0.285714,0.444444,0.285714,0.545455,-1.807355,0.452267,0.019531,0.375000,0.250000,0.125000,0.222222,0.0,0.058824,0.292758,0.881000,0.593750,0.724138,0.272727,0.000000,0.375000,0.0,0.000000,0.687500,0.0,0.0
17,eigenmanndaniela,rosoffmeg,0.000000,0.0,0.062500,0.062500,0.395833,0.395833,0.447917,0.312500,0.000000,0.136364,0.240000,0.136364,0.333333,-2.874469,0.250000,0.009766,0.187500,0.125000,0.062500,0.160000,0.0,0.058824,0.174993,0.817973,0.656250,0.724138,0.272727,0.000000,0.281250,0.0,0.000000,0.562500,0.0,0.0



person_245c
**********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
93,[hrsg.:] schweizerische gesellschaft für bildu...,"voltaire ; introd., notes , bibliogr., chronol...",0.006645,0.0,0.132890,0.136213,0.541068,0.541068,0.548457,0.176080,0.075758,0.191558,0.321526,0.191558,0.893939,-2.384143,0.418598,0.001485,0.196013,0.136213,0.009967,0.092643,0.0,0.019920,0.282030,0.957093,0.173077,0.243697,0.145251,0.000000,0.209302,0.0,0.000000,0.219269,0.0,0.0
23,[hrsg.:] schweizerische gesellschaft für bildu...,hrsg. schweizerische gesellschaft für bildungs...,0.066225,0.0,0.884106,0.884106,0.821598,0.821598,0.821598,0.912252,0.830565,0.908228,0.951907,0.908228,0.953488,-0.138874,0.951908,0.001628,0.950331,0.940397,0.238411,0.941957,0.0,0.405512,0.558540,0.988137,0.740385,0.865546,0.826816,0.000000,0.890728,0.0,0.003311,0.996689,0.0,0.0
5,[hrsg.:] schweizerische gesellschaft für bildu...,w.a. mozart ; libretto emanuel schikaneder,0.006645,0.0,0.096346,0.099668,0.527880,0.527880,0.536019,0.117940,0.119048,0.124590,0.221574,0.124590,0.904762,-3.004738,0.337968,0.001468,0.126246,0.099668,0.013289,0.064140,0.0,0.003984,0.232013,0.962976,0.149038,0.226891,0.117318,0.000000,0.176080,0.0,0.000000,0.139535,0.0,0.0
95,[hrsg.:] schweizerische gesellschaft für bildu...,wolfgang amadeus mozart ; [text von emanuel sc...,0.019934,0.0,0.172757,0.182724,0.583696,0.583696,0.594197,0.255814,0.137255,0.300000,0.461538,0.300000,0.911765,-1.736966,0.530762,0.001545,0.308970,0.189369,0.013289,0.188586,0.0,-0.003984,0.342676,0.963808,0.168269,0.260504,0.150838,0.000000,0.270764,0.0,0.000000,0.338870,0.0,0.0
137,[hrsg.:] schweizerische gesellschaft für bildu...,von emanuel schickaneder ; musik von j. mozart,0.006645,0.0,0.112957,0.112957,0.484111,0.484111,0.501654,0.132890,0.130435,0.126623,0.224784,0.126623,0.847826,-2.981384,0.331438,0.001479,0.129568,0.112957,0.013289,0.126801,0.0,0.000000,0.233418,0.961036,0.144231,0.226891,0.122905,0.000000,0.187708,0.0,0.000000,0.152824,0.0,0.0
27,[hrsg.:] schweizerische gesellschaft für bildu...,von beatrice käser,0.009967,0.0,0.049834,0.049834,0.480682,0.480682,0.486569,0.054817,0.111111,0.052805,0.100313,0.052805,0.888889,-4.243174,0.217370,0.001187,0.053156,0.049834,0.013289,0.087774,0.0,0.000000,0.134509,0.884746,0.153846,0.193277,0.061453,0.000000,0.121262,0.0,0.000000,0.059801,0.0,0.0
162,[hrsg.:] schweizerische gesellschaft für bildu...,sigrid kessler...[et al.],0.003322,0.0,0.066445,0.066445,0.499515,0.499515,0.503847,0.074751,0.080000,0.075908,0.141104,0.075908,0.920000,-3.719612,0.265140,0.001098,0.076412,0.066445,0.009967,0.067485,0.0,0.007968,0.159762,0.871815,0.125000,0.210084,0.078212,0.166667,0.137874,0.0,0.000000,0.083056,0.0,0.0
139,[hrsg.:] schweizerische gesellschaft für bildu...,von w. a. mozart ; klavierauszug revidiert von...,0.006645,0.0,0.122924,0.126246,0.510233,0.510233,0.525794,0.164452,0.129032,0.186275,0.314050,0.186275,0.919355,-2.424498,0.417249,0.001347,0.189369,0.126246,0.013289,0.148760,0.0,0.003984,0.268529,0.954475,0.149038,0.243697,0.150838,0.000000,0.205980,0.0,0.000000,0.205980,0.0,0.0
114,[hrsg.:] schweizerische gesellschaft für bildu...,sigrid kessler ... [et al.] ; [ill.: rolf schm...,0.006645,0.0,0.109635,0.109635,0.523878,0.523878,0.528542,0.137874,0.120000,0.147059,0.256410,0.147059,0.900000,-2.765535,0.366813,0.001374,0.149502,0.109635,0.013289,0.062678,0.0,0.019920,0.244309,0.941572,0.168269,0.226891,0.134078,0.166667,0.181063,0.0,0.000000,0.166113,0.0,0.0
163,[hrsg.:] schweizerische gesellschaft für bildu...,w. a. mozart,0.000000,0.0,0.033223,0.033223,0.529858,0.529858,0.538524,0.036545,0.083333,0.036424,0.070288,0.036424,0.916667,-4.778973,0.183029,0.000568,0.036545,0.033223,0.003322,0.031949,0.0,0.003984,0.086298,0.769049,0.1


person_700
**********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
138,"gröbliroland, nikolaus",müllhaupthans heinrich,0.045455,0.0,0.090909,0.181818,0.494949,0.494949,0.540404,0.522727,0.090909,0.375000,0.545455,0.375000,0.545455,-1.415037,0.545455,0.014463,0.545455,0.318182,0.090909,0.318182,0.0,0.043478,0.327752,0.906811,0.571429,0.636364,0.206897,0.000000,0.227273,0.0,0.000000,1.000000,0.0,0.0
20,"gröbliroland, nikolaus","schlöndorffvolker, wurlitzerrudy, frischmax",0.046512,0.0,0.209302,0.209302,0.574789,0.574789,0.616015,0.360465,0.136364,0.382979,0.553846,0.382979,0.818182,-1.384664,0.585230,0.020661,0.418605,0.209302,0.046512,0.276923,0.0,0.045455,0.310843,0.880923,0.500000,0.488372,0.140000,0.000000,0.372093,0.0,0.000000,0.511628,0.0,0.0
39,"gröbliroland, nikolaus","schikanederemanuel, soldankurt",0.033333,0.0,0.233333,0.233333,0.571717,0.571717,0.595354,0.483333,0.136364,0.444444,0.615385,0.444444,0.727273,-1.169925,0.622799,0.019628,0.533333,0.333333,0.066667,0.307692,0.0,0.033333,0.401996,0.937309,0.551020,0.567568,0.166667,0.166667,0.316667,0.0,0.000000,0.733333,0.0,0.0
124,"gröbliroland, nikolaus",schusterhans-peter,0.090909,0.0,0.090909,0.090909,0.468013,0.468013,0.528620,0.454545,0.055556,0.176471,0.300000,0.176471,0.333333,-2.502500,0.301511,0.008264,0.272727,0.181818,0.090909,0.200000,0.0,0.043478,0.197190,0.814530,0.448980,0.636364,0.214286,0.000000,0.227273,0.0,0.000000,0.818182,0.0,0.0
84,"gröbliroland, nikolaus","kesslersigrid, ritschardcharlotte",0.060606,0.0,0.212121,0.212121,0.544012,0.544012,0.574315,0.439394,0.090909,0.375000,0.545455,0.375000,0.681818,-1.415037,0.556702,0.017562,0.454545,0.242424,0.090909,0.290909,0.0,0.029412,0.377838,0.931062,0.510204,0.589744,0.194444,0.166667,0.348485,0.0,0.000000,0.666667,0.0,0.0
42,"gröbliroland, nikolaus","moehnheinz, schikanederemanuel, grubergernot",0.045455,0.0,0.136364,0.159091,0.540909,0.540909,0.554545,0.318182,0.045455,0.404255,0.575758,0.404255,0.863636,-1.306661,0.610683,0.021694,0.431818,0.181818,0.045455,0.151515,0.0,0.045455,0.364934,0.927416,0.517241,0.488372,0.180000,0.000000,0.272727,0.0,0.000000,0.500000,0.0,0.0
32,"gröbliroland, nikolaus","heilmannuwe, josumi, krausmichael, leitnerlott...",0.000000,0.0,0.145455,0.145455,0.538437,0.538437,0.543892,0.172727,0.181818,0.178571,0.303030,0.178571,0.909091,-2.485427,0.406558,0.020661,0.181818,0.145455,0.027273,0.242424,0.0,0.009709,0.255915,0.908876,0.295918,0.295775,0.123596,0.000000,0.250000,0.0,0.000000,0.200000,0.0,0.0
26,"gröbliroland, nikolaus","levinejames, hockneydavid",0.040000,0.0,0.160000,0.200000,0.601072,0.601072,0.635254,0.480000,0.045455,0.382353,0.553191,0.382353,0.590909,-1.387023,0.554322,0.016529,0.520000,0.280000,0.080000,0.297872,0.0,0.038462,0.345014,0.907163,0.510204,0.657143,0.193548,0.000000,0.300000,0.0,0.000000,0.880000,0.0,0.0
122,"gröbliroland, nikolaus","scherrera.u., ledergerberb., von wylv., bönij....",0.002845,0.0,0.029872,0.031294,0.525583,0.525583,0.525583,0.030583,0.090909,0.031294,0.060690,0.031294,1.000000,-4.997949,0.176902,0.022727,0.031294,0.029872,0.005690,0.055172,0.0,-0.005474,0.115956,0.879130,0.081522,0.115207,0.039326,0.000000,0.109531,0.0,0.000000,0.031294,0.0,0.0
17,"gröbliroland, nikolaus","fesslermichel, jacquetluc",0.080000,0.0,0.200000,0.200000,0.478586,0.478586,0.538404,0.540000,0.090909,0.270270,0.425532,0.270270,0.454545,-1.887525,0.426401,0.013430,0.400000,0.280000,0.080000,0.255319,0.0,0.038462,0.290112,0.896905,0.469388,0.657143,0.193548,0.166667,0.360000,0.0,0.000000,0.880000,0.0,0.0


### pubinit

In [16]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'pubinit_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
72,alber,bei a.h. hirsch,0.000000,0.0,0.133333,0.133333,0.488889,0.488889,0.515556,0.233333,0.0,0.250000,0.400000,0.250000,0.8,-2.000000,0.461880,0.08,0.266667,0.200000,0.133333,0.300000,0.0,0.062500,0.195675,0.746472,0.638889,0.793103,0.285714,0.000000,0.266667,0.000000,0.000000,0.333333,0.0,0.0
17,alber,"universitätsverlag, klett + balmer",0.000000,0.0,0.147059,0.147059,0.472549,0.472549,0.495490,0.147059,0.8,0.147059,0.256410,0.147059,1.0,-2.765535,0.383482,0.10,0.147059,0.147059,0.058824,0.205128,0.0,0.058824,0.146564,0.677262,0.438596,0.589744,0.150000,0.000000,0.191176,0.000000,0.058824,0.147059,0.0,0.0
67,alber,servizio topografico federale,0.000000,0.0,0.103448,0.103448,0.489655,0.489655,0.560000,0.137931,0.2,0.133333,0.235294,0.133333,0.8,-2.906891,0.332182,0.08,0.137931,0.103448,0.068966,0.176471,0.0,0.066667,0.133480,0.686192,0.500000,0.621622,0.171429,0.000000,0.137931,0.000000,0.000000,0.172414,0.0,0.0
76,alber,libération,0.200000,0.0,0.200000,0.300000,0.633333,0.633333,0.693333,0.350000,0.2,0.363636,0.533333,0.363636,0.8,-1.459432,0.565685,0.08,0.400000,0.300000,0.100000,0.133333,0.0,0.090909,0.248894,0.777217,0.666667,0.851852,0.352941,0.166667,0.250000,0.000000,0.000000,0.500000,0.0,0.0
88,alber,de fer,0.000000,0.0,0.333333,0.333333,0.577778,0.577778,0.614444,0.583333,0.4,0.222222,0.363636,0.222222,0.4,-2.169925,0.365148,0.04,0.333333,0.333333,0.333333,0.363636,0.0,0.142857,0.234189,0.821272,0.793103,0.920000,0.500000,0.250000,0.416667,0.000000,0.333333,0.833333,0.0,0.0
93,alber,peters,0.333333,0.0,0.333333,0.333333,0.577778,0.577778,0.614444,0.583333,0.2,0.222222,0.363636,0.222222,0.4,-2.169925,0.365148,0.04,0.333333,0.333333,0.333333,0.363636,0.0,0.142857,0.234189,0.821272,0.851852,0.920000,0.500000,0.000000,0.333333,0.000000,0.000000,0.833333,0.0,0.0
74,alber,p. reclam jun.,0.000000,0.0,0.071429,0.071429,0.493651,0.493651,0.520794,0.214286,0.0,0.266667,0.421053,0.266667,0.8,-1.906891,0.478091,0.08,0.285714,0.071429,0.071429,0.105263,0.0,0.066667,0.182653,0.713728,0.666667,0.793103,0.300000,0.000000,0.142857,0.000000,0.000000,0.357143,0.0,0.0
47,alber,atlantis musikbuch-verlag,0.040000,0.0,0.200000,0.200000,0.493333,0.493333,0.541333,0.200000,0.0,0.200000,0.333333,0.200000,1.0,-2.321928,0.447214,0.10,0.200000,0.200000,0.080000,0.333333,0.0,0.038462,0.152457,0.664969,0.500000,0.657143,0.193548,0.333333,0.240000,0.040000,0.000000,0.200000,0.0,0.0
4,alber,deutsche grammophon gesellschaft,0.000000,0.0,0.062500,0.062500,0.453472,0.453472,0.453472,0.109375,0.0,0.121212,0.216216,0.121212,0.8,-3.044394,0.316228,0.08,0.125000,0.062500,0.031250,0.108108,0.0,0.030303,0.119153,0.660531,0.460000,0.567568,0.162162,0.000000,0.250000,0.000000,0.000000,0.156250,0.0,0.0
68,alber,archiati-verlag,0.066667,0.0,0.200000,0.200000,0.511111,0.511111,0.591111,0.266667,0.0,0.250000,0.400000,0.250000,0.8,-2.000000,0.461880,0.08,0.266667,0.200000,0.133333,0.300000,0.0,0.062500,0.205867,0.754291,0.605263,0.793103,0.285714,0.333333,0.333333,0.066667,0.000000,0.333333,0.0,0.0


### scale

In [17]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'scale_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
2,50000,50000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.000000,1.000000,1.00,1.000000,1.000000,1.000000,1.000000,0.0,0.25,0.585786,1.000000,1.000000,0.920000,0.666667,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0
3,50000,50 000 8 10 8 35 45 55 46 05,0.142857,0.0,0.178571,0.178571,0.726190,0.780952,0.726190,0.178571,0.2,0.178571,0.303030,0.178571,1.0,-2.485427,0.422577,0.10,0.178571,0.178571,0.107143,0.303030,0.0,0.00,0.152319,0.509693,0.594595,0.675676,0.233333,0.0,0.285714,0.071429,0.000000,0.178571,0.0,0.0
0,50000,,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,-inf,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.25,0.000000,0.580744,0.000000,0.360000,0.666667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,50000,100000,0.666667,1.0,0.666667,0.833333,0.822222,0.822222,0.822222,0.750000,0.8,0.571429,0.727273,0.571429,0.8,-0.807355,0.730297,0.08,0.666667,0.666667,0.666667,0.727273,0.0,0.25,0.381966,0.874656,0.884615,0.920000,0.666667,0.5,0.833333,0.000000,0.666667,0.833333,0.0,0.0


### ttlfull

Attribute $\texttt{ttlfull}$ consists of two different representations of data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). Both representations will be investigated below.

In [18]:
ttlfull_representations = ['245', '246']

for tf in ttlfull_representations :
    df_string_pairs = daf.string_pair_list(df_feature_base, 'ttlfull_'+tf+'_x')

    print('\nttlfull_'+tf+'\n***********')
    for algorithm in tedi_algorithms :
        if algorithm not in ['Gotoh', 'ArithNCD', 'Editex'] : # Some (very long string) samples fail with Editex
            daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

    display(df_string_pairs.sample(n=num_of_samples(df_string_pairs)))


ttlfull_245
***********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Prefix,Postfix,Length,Identity,Matrix
28,"die zauberflöte, eine grosse oper in zwey aufz...","die zauberflöte, il flauto magico : oper in zw...",0.333333,0.0,0.592593,0.611111,0.748604,0.849162,0.863029,0.731481,0.500000,0.600000,0.750000,0.600000,0.780000,-0.736966,0.750555,0.0092,0.722222,0.685185,0.314815,0.711538,0.0,0.181818,0.494201,0.971277,0.623377,0.632653,0.573770,0.666667,0.314815,0.037037,0.925926,0.0,0.0
118,"die zauberflöte, eine grosse oper in zwey aufz...","bonne chance !, cours de langue française prem...",0.041096,0.0,0.260274,0.287671,0.615982,0.615982,0.629461,0.472603,0.180000,0.518519,0.682927,0.518519,0.840000,-0.947533,0.695189,0.0086,0.575342,0.315068,0.041096,0.260163,0.0,-0.014493,0.396635,0.955673,0.352941,0.355932,0.289474,0.000000,0.000000,0.000000,0.684932,0.0,0.0
32,"die zauberflöte, eine grosse oper in zwey aufz...","die zauberflöte, oper in 2 aufzügen",0.360000,0.0,0.680000,0.680000,0.815182,0.889109,0.892024,0.690000,0.542857,0.666667,0.800000,0.666667,0.971429,-0.584963,0.812755,0.0092,0.680000,0.680000,0.340000,0.800000,0.0,0.137255,0.485808,0.987366,0.666667,0.702128,0.620690,1.000000,0.340000,0.180000,0.700000,0.0,0.0
141,"die zauberflöte, eine grosse oper in zwey aufz...",combined use of mycobacterium tuberculosis-spe...,0.000000,0.0,0.210526,0.218045,0.572731,0.572731,0.583738,0.293233,0.120000,0.307143,0.469945,0.307143,0.860000,-1.703018,0.527300,0.0088,0.323308,0.225564,0.030075,0.207650,0.0,0.030303,0.351093,0.955282,0.290598,0.308642,0.215686,0.000000,0.000000,0.000000,0.375940,0.0,0.0
30,"die zauberflöte, eine grosse oper in zwey aufz...","die zauberflöte, oper in 2 aufzügen : [kv 620]",0.380000,0.0,0.460000,0.460000,0.760129,0.856077,0.863590,0.670000,0.369565,0.600000,0.750000,0.600000,0.782609,-0.736966,0.750652,0.0092,0.720000,0.680000,0.340000,0.708333,0.0,0.137255,0.457553,0.941074,0.621622,0.659574,0.620690,0.500000,0.340000,0.000000,0.920000,0.0,0.0
47,"die zauberflöte, eine grosse oper in zwey aufz...","die zauberflöte, textbuch",0.340000,0.0,0.380000,0.380000,0.713333,0.828000,0.835200,0.440000,0.160000,0.339286,0.506667,0.339286,0.760000,-1.559427,0.537401,0.0074,0.380000,0.380000,0.340000,0.506667,0.0,0.058824,0.341305,0.932851,0.583333,0.531915,0.362069,0.500000,0.340000,0.000000,0.500000,0.0,0.0
62,"die zauberflöte, eine grosse oper in zwey aufz...","bonne chance!, cours de langue française : tro...",0.020833,0.0,0.239583,0.260417,0.594556,0.594556,0.615848,0.380208,0.160000,0.390476,0.561644,0.390476,0.820000,-1.356694,0.591784,0.0084,0.427083,0.270833,0.031250,0.205479,0.0,0.000000,0.348195,0.944755,0.320388,0.342466,0.255319,0.000000,0.000000,0.000000,0.520833,0.0,0.0
146,"die zauberflöte, eine grosse oper in zwey aufz...","the complete works of voltaire, siècle de loui...",0.071429,0.0,0.160714,0.196429,0.567460,0.567460,0.605317,0.526786,0.100000,0.413333,0.584906,0.413333,0.620000,-1.274622,0.585845,0.0076,0.553571,0.339286,0.053571,0.301887,0.0,-0.018182,0.385094,0.939028,0.421053,0.372549,0.338710,0.000000,0.000000,0.000000,0.892857,0.0,0.0
3,"die zauberflöte, eine grosse oper in zwey aufz...","die zauberflöte, oper in zwei aufzügen : kv 620",0.380000,0.0,0.560000,0.560000,0.792021,0.875213,0.877689,0.700000,0.425532,0.701754,0.824742,0.701754,0.851064,-0.510962,0.825137,0.0094,0.800000,0.740000,0.340000,0.762887,0.0,0.176471,0.492151,0.958305,0.662162,0.744681,0.672414,0.500000,0.340000,0.000000,0.940000,0.0,0.0
24,"die zauberflöte, eine grosse oper in zwey aufz...","homo faber, originaltext, interpretation, biog...",0.056180,0.0,0.269663,0.269663,0.583442,0.583442,0.605307,0.410112,0.140000,0.404040,0.575540,0.404040,0.800000,-1.307429,0.599625,0.0086,0.449438,0.314607,0.033708,0.273381,0.0


ttlfull_246
***********
Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Prefix,Postfix,Length,Identity,Matrix
10,medizinische informatik - kommunikation von ge...,flûte enchanté,0.000000,0.0,0.036697,0.036697,0.492391,0.492391,0.492391,0.039755,0.000000,0.036474,0.070381,0.036474,0.857143,-4.776981,0.177355,0.000669,0.036697,0.036697,0.006116,0.052786,0.0,0.003401,0.086734,0.767467,0.135371,0.162791,0.054187,0.000000,0.000000,0.000000,0.042813,0.0,0.0
35,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,0.318296,0.0,0.711779,0.714286,0.797409,0.878445,0.880782,0.764411,0.645260,0.720379,0.837466,0.720379,0.929664,-0.473172,0.841615,0.001501,0.761905,0.721805,0.233083,0.752066,0.0,0.359420,0.531382,0.985092,0.575188,0.687075,0.626609,0.833333,0.233083,0.002506,0.819549,0.0,0.0
0,medizinische informatik - kommunikation von ge...,,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.003401,0.000000,0.176137,0.000000,0.069767,0.029557,0.000000,0.000000,0.000000,0.000000,0.0,0.0
18,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,0.376147,0.0,0.792049,0.795107,0.840267,0.904160,0.910384,0.842508,0.767123,0.794203,0.885299,0.794203,0.938356,-0.332420,0.886717,0.001520,0.837920,0.807339,0.284404,0.849758,0.0,0.363946,0.554167,0.993757,0.646288,0.767442,0.719212,0.833333,0.284404,0.003058,0.892966,0.0,0.0
20,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,0.382263,0.0,0.565749,0.584098,0.815168,0.889101,0.895737,0.747706,0.500000,0.750000,0.857143,0.750000,0.861111,-0.415037,0.857152,0.001501,0.853211,0.712538,0.284404,0.700461,0.0,0.299320,0.547434,0.984944,0.541485,0.643411,0.581281,0.833333,0.284404,0.003058,0.990826,0.0,0.0
34,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,0.295195,0.0,0.526316,0.533181,0.771310,0.862786,0.865352,0.635011,0.360856,0.646552,0.785340,0.646552,0.917431,-0.629162,0.793609,0.001501,0.686499,0.576659,0.212815,0.596859,0.0,0.249357,0.510699,0.982897,0.400685,0.533742,0.447876,0.833333,0.212815,0.002288,0.748284,0.0,0.0
28,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,0.299539,0.0,0.534562,0.546083,0.769206,0.861524,0.864097,0.641705,0.376147,0.668860,0.801577,0.668860,0.932722,-0.580225,0.809620,0.001501,0.702765,0.582949,0.216590,0.628121,0.0,0.254593,0.508357,0.986534,0.406143,0.539394,0.452830,0.833333,0.216590,0.002304,0.753456,0.0,0.0
15,medizinische informatik - kommunikation von ge...,die zauberflöte,0.003058,0.0,0.039755,0.039755,0.492049,0.492049,0.499021,0.042813,0.000000,0.042683,0.081871,0.042683,0.933333,-4.550197,0.199898,0.000762,0.042813,0.039755,0.012232,0.046784,0.0,0.003401,0.104216,0.804296,0.126638,0.178295,0.059113,0.166667,0.000000,0.000000,0.045872,0.0,0.0
21,medizinische informatik - kommunikation von ge...,medizinische informatik - kommunikation von ge...,0.366972,0.0,0.792049,0.798165,0.834625,0.900775,0.906191,0.845566,0.766102,0.782235,0.877814,0.782235,0.925424,-0.354326,0.878978,0.001501,0.834862,0.816514,0.284404,0.858521,0.0,0.367347,0.543424,0.988001,0.646288,0.751938,0.699507,0.833333,0.284404,0.003058,0.902141,0.0,0.0
9,medizinische informatik - kommunikation von ge...,"die zauberflöte, ausgabe für gesang und klavier",0.012232,0.0,0.103976,0.103976,0.550206,0.550206,0.555073,0.123853,0.042553,0.136778,0.240642,0.136778,0.957447,-2.870091,0.362986,0.000991,0.137615,0.103976,0.015291,0.139037,0.0,0.017007,0.207516,0.882024,0.157205,0.224806,0.137931,0.166667,0.000000,0.000000,0.143731,0.0,0.0


### volumes

In [19]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'volumes_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['Gotoh', 'ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=num_of_samples(df_string_pairs))

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
Jaro
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,Jaro,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
67,304,1 32,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.375000,0.000000,0.166667,0.285714,0.166667,0.333333,-2.584963,0.288675,0.055556,0.250000,0.250000,0.250000,0.285714,0.00,0.200000,0.146447,0.687774,0.814815,0.913043,0.600000,0.000000,0.000000,0.000000,0.000000,0.750000,0.0,0.0
1,304,1 0 169,0.000000,0.0,0.142857,0.142857,0.492063,0.492063,0.492063,0.285714,0.000000,0.111111,0.200000,0.111111,0.333333,-3.169925,0.218218,0.055556,0.142857,0.142857,0.142857,0.200000,0.00,0.125000,0.100505,0.648643,0.750000,0.840000,0.461538,0.000000,0.142857,0.000000,0.000000,0.428571,0.0,0.0
82,304,1 33,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.375000,0.000000,0.166667,0.285714,0.166667,0.333333,-2.584963,0.288675,0.055556,0.250000,0.250000,0.250000,0.285714,0.25,0.200000,0.199801,0.757023,0.800000,0.913043,0.600000,0.000000,0.250000,0.000000,0.000000,0.750000,0.0,0.0
44,304,1 109,0.000000,0.0,0.200000,0.200000,0.000000,0.000000,0.000000,0.400000,0.333333,0.142857,0.250000,0.142857,0.333333,-2.807355,0.258199,0.055556,0.200000,0.200000,0.200000,0.250000,0.00,0.166667,0.132705,0.686838,0.814815,0.920000,0.545455,0.000000,0.200000,0.000000,0.000000,0.600000,0.0,0.0
56,304,365,0.333333,1.0,0.333333,0.333333,0.555556,0.555556,0.555556,0.666667,0.333333,0.200000,0.333333,0.200000,0.333333,-2.321928,0.333333,0.055556,0.333333,0.333333,0.333333,0.333333,0.00,0.250000,0.195262,0.742098,0.913043,0.913043,0.666667,0.333333,0.333333,0.333333,0.000000,1.000000,0.0,0.0
40,304,26,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.488608,0.869565,0.913043,0.666667,0.000000,0.000000,0.000000,0.000000,0.666667,0.0,0.0
35,304,214,0.333333,1.0,0.333333,0.333333,0.555556,0.555556,0.555556,0.666667,0.333333,0.200000,0.333333,0.200000,0.333333,-2.321928,0.333333,0.055556,0.333333,0.333333,0.333333,0.333333,0.00,0.250000,0.195262,0.742098,0.913043,0.913043,0.666667,0.333333,0.333333,0.000000,0.333333,1.000000,0.0,0.0
94,304,108,0.333333,1.0,0.333333,0.333333,0.555556,0.555556,0.555556,0.666667,0.333333,0.200000,0.333333,0.200000,0.333333,-2.321928,0.333333,0.055556,0.333333,0.333333,0.333333,0.333333,0.00,0.250000,0.195262,0.742098,0.913043,0.913043,0.666667,0.333333,0.333333,0.000000,0.000000,1.000000,0.0,0.0
9,304,2 73 17 70 26,0.000000,0.0,0.153846,0.153846,0.470085,0.470085,0.470085,0.192308,0.000000,0.142857,0.250000,0.142857,0.666667,-2.807355,0.320256,0.111111,0.153846,0.153846,0.076923,0.250000,0.00,0.071429,0.128093,0.651416,0.677419,0.793103,0.315789,0.000000,0.153846,0.000000,0.000000,0.230769,0.0,0.0
6,304,64,0.000000,1.0,0.333333,0.333333,0.000000,0.000000,0.000000,0.500000,0.500000,0.250000,0.400000,0.250000,0.500000,-2.000000,0.408248,0.055556,0.333333,0.333333,0.333333,0.400000,0.00,0.250000,0.195262,0.643350,0.869565,0.913043,0.666667,0.000000,0.333333,0.000000,0.333333,0.666667,0.0,0.0
